We work with polynomials whose coefficients are integers modulo $p$.

In [85]:
def poly_to_string(poly_coeffs):
    '''
    Converts a list of polynomial coefficients into a string.
    '''
    poly_coeffs = _trim(list(poly_coeffs))
    if not poly_coeffs:
        return "0"

    terms = []
    for i, coeff in enumerate(poly_coeffs):
        if coeff == 0:
            continue

        # Coefficient part
        if i > 0 and coeff == 1:
            coeff_str = ""
        else:
            coeff_str = str(coeff)

        # Variable part
        if i == 0:
            var_str = ""
        elif i == 1:
            var_str = "x"
        else:
            var_str = f"x^{i}"

        if i > 0 and coeff == 1:
            terms.append(var_str)
        elif coeff_str:
            terms.append(f"{coeff_str}{var_str}")

    return " + ".join(reversed(terms)).replace(" + -", " - ")

def mod_inverse(n, modulus):
    '''
    Computes the modular inverse of n modulo modulus.
    '''
    return pow(n, -1, modulus)

def _trim(poly):
    '''
    Removes leading zero coefficients from a polynomial list.
    '''
    while len(poly) > 0 and poly[-1] == 0:
        poly.pop()
    return poly

def poly_divmod(dividend, divisor, p):
    '''
    Computes quotient and remainder for polynomial division in Z_p.
    '''
    dividend = _trim(list(dividend))
    divisor = _trim(list(divisor))

    if not divisor:
        raise ZeroDivisionError("Polynomial division by zero")

    deg_divisor = len(divisor) - 1
    deg_dividend = len(dividend) - 1

    if deg_dividend < deg_divisor:
        return ([0], dividend)

    # Make a copy of the dividend to use as the changing remainder
    remainder = list(dividend)
    quotient = [0] * (deg_dividend - deg_divisor + 1)

    # Get the modular inverse of the divisor's leading coefficient
    inv_lead_divisor = mod_inverse(divisor[-1], p)

    # Perform long division, starting from the highest degree
    for i in range(deg_dividend, deg_divisor - 1, -1):
        if len(remainder) - 1 < i:
            continue

        q_coeff = (remainder[-1] * inv_lead_divisor) % p
        deg_q_term = len(remainder) - 1 - deg_divisor
        quotient[deg_q_term] = q_coeff

        # Subtract (q_term * divisor) from the remainder
        for j in range(deg_divisor + 1):
            sub_index = deg_q_term + j
            sub_val = (q_coeff * divisor[j]) % p
            remainder[sub_index] = (remainder[sub_index] - sub_val + p) % p

        remainder = _trim(remainder)

    return (_trim(quotient), _trim(remainder))

def poly_gcd(p1, p2, p):
    '''
    Computes the GCD of two polynomials in Z_p using the Euclidean algorithm.
    '''
    a = _trim(list(p1))
    b = _trim(list(p2))

    while b:  # Loop continues as long as b is not the zero polynomial ([])
        _, r = poly_divmod(a, b, p)
        a = b
        b = r

    # Make the final GCD monic (leading coefficient is 1)
    if not a:
        return [0]

    lead_coeff = a[-1]
    inv_lead = mod_inverse(lead_coeff, p)
    monic_gcd = [(c * inv_lead) % p for c in a]

    return monic_gcd

def poly_mul_mod(p1, p2, mod_poly, p):
    '''
    Computes (p1 * p2) mod mod_poly in Z_p.
    '''
    p1, p2 = list(p1), list(p2)
    deg1, deg2 = len(p1) - 1, len(p2) - 1
    prod = [0] * (deg1 + deg2 + 1)
    for i in range(deg1 + 1):
        for j in range(deg2 + 1):
            prod[i+j] = (prod[i+j] + p1[i] * p2[j]) % p
    _, rem = poly_divmod(prod, mod_poly, p)
    return rem

def poly_pow_mod(base, exp, mod_poly, p):
    '''
    Computes (base^exp) mod mod_poly in Z_p using repeated squaring.
    '''
    base = list(base)
    res = [1] # Represents the polynomial 1
    while exp > 0:
        if exp % 2 == 1:
            res = poly_mul_mod(res, base, mod_poly, p)
        base = poly_mul_mod(base, base, mod_poly, p)
        exp //= 2
    return res

def evaluate_poly(poly_coeffs, x, p):
    '''
    Evaluates a polynomial f(x) at a given value x modulo p.
    Uses Horner's method for efficiency.
    '''
    result = 0
    for coeff in reversed(poly_coeffs):
        result = (result * x + coeff) % p
    return result

In [86]:
p1 = 109
poly1_1 = [4, 12, 8, 1]   # x^3 + 8x^2 + 12x + 4
poly1_2 = [10, 2, 6, 1]   # x^3 + 6x^2 + 2x + 10
gcd1 = poly_gcd(poly1_1, poly1_2, p1)
print(f"--- Example 1 (p = {p1}) ---")
print(f"gcd({poly_to_string(poly1_1)}, {poly_to_string(poly1_2)})")
print(f"Result: {poly_to_string(gcd1)}\n")

p2 = 131
poly2_1 = [8, 6, 2, 1]   # x^3 + 2x^2 + 6x + 8
poly2_2 = [2, 1, 11, 1]  # x^3 + 11x^2 + x + 2
gcd2 = poly_gcd(poly2_1, poly2_2, p2)
print(f"--- Example 2 (p = {p2}) ---")
print(f"gcd({poly_to_string(poly2_1)}, {poly_to_string(poly2_2)})")
print(f"Result: {poly_to_string(gcd2)}\n")

p3 = 157
poly3_1 = [1, 7, 3, 1]   # x^3 + 3x^2 + 7x + 1
poly3_2 = [12, 4, 3, 1]  # x^3 + 3x^2 + 4x + 12
gcd3 = poly_gcd(poly3_1, poly3_2, p3)
print(f"--- Example 3 (p = {p3}) ---")
print(f"gcd({poly_to_string(poly3_1)}, {poly_to_string(poly3_2)})")
print(f"Result: {poly_to_string(gcd3)}")

--- Example 1 (p = 109) ---
gcd(x^3 + 8x^2 + 12x + 4, x^3 + 6x^2 + 2x + 10)
Result: 1

--- Example 2 (p = 131) ---
gcd(x^3 + 2x^2 + 6x + 8, x^3 + 11x^2 + x + 2)
Result: x^2 + 14x + 43

--- Example 3 (p = 157) ---
gcd(x^3 + 3x^2 + 7x + 1, x^3 + 3x^2 + 4x + 12)
Result: x + 101


We implement the Cantor-Zassenhaus algorithm, a probabilistic method for factoring polynomials over finite fields. We will apply it to the specific problem of finding square roots, which is equivalent to factoring the polynomial $f(x) = x^2 - a$.

To compute the roots of a polynomial $f(x) \bmod p$, we first compute $\gcd(f(x), x^p - x)$. This reduces us to the case where $f(x)$ is a product of distinct linear factors. We then pick a small integer $v$ at random and attempt to factor f(x) by computing $\gcd(f(x), g(x))$ where $g(x) = (x + v)^{(p−1)/2} - 1$. This will be successful unless the numbers $\alpha + v$ for $\alpha$ running over the roots of $f(x)$ are either all quadratic residues, or all non-residues. If unsuccessful we try another value of $v$.


In [87]:
def find_square_root_cz(a, p):
    '''
    Computes a square root of 'a' modulo 'p' using the Cantor-Zassenhaus algorithm.
    '''
    a %= p
    # The polynomial we want to factor is f(x) = x^2 - a
    f_x = [(-a + p) % p, 0, 1]

    # The algorithm fails if f(x) has no roots, so we check first.
    if pow(a, (p - 1) // 2, p) == p - 1:
        print(f"{a} is not a quadratic residue modulo {p}.")
        return None
    if a == 0:
        return 0

    # Loop until we find a factor
    v = 1
    while True:
        print(f"Trying with v = {v}...")
        # Construct g(x) = (x + v)^((p-1)/2) - 1
        # We compute (x+v)^exp mod f(x) to keep the degree low.
        base_poly = [v, 1]  # Represents x + v
        exp = (p - 1) // 2

        # Avoids high-degree polynomials
        pow_poly = poly_pow_mod(base_poly, exp, f_x, p)

        # g(x) = pow_poly - 1
        g_x = list(pow_poly)
        g_x[0] = (g_x[0] - 1 + p) % p

        # Compute the GCD
        h_x = poly_gcd(f_x, g_x, p)

        # Check if we found a non-trivial factor
        # A trivial factor is 1 (deg < 1) or f_x itself (deg > 1)
        if len(h_x) - 1 == 1:
            # The factor is of the form c*(x - root).
            # The root is -c_0 * c_1^(-1)
            root = (-h_x[0] * mod_inverse(h_x[1], p) + p) % p
            print(f"Found factor h(x) = {poly_to_string(h_x)}")
            return root

        print(f"Failed, gcd(f(x), g(x)) was trivial: {poly_to_string(h_x)}")
        v += 1

p = 101
a = 22
print(f"Finding square roots of {a} modulo {p} using Cantor-Zassenhaus:\n")
root = find_square_root_cz(a, p)
if root is not None:
    print(f"\nA square root of {a} is {root}.")
    print(f"The other root is {p - root}.")
    print(f"Verification: {root}^2 mod {p} = {pow(root, 2, p)}")
    print(f"Verification: {p - root}^2 mod {p} = {pow(root, 2, p)}")

Finding square roots of 22 modulo 101 using Cantor-Zassenhaus:

Trying with v = 1...
Failed, gcd(f(x), g(x)) was trivial: 1
Trying with v = 2...
Found factor h(x) = x + 27

A square root of 22 is 74.
The other root is 27.
Verification: 74^2 mod 101 = 22
Verification: 27^2 mod 101 = 22


The algorithm requires computing $g(x) = (x + v)^{(p-1)/2} - 1$. A naive implementation would calculate the full polynomial which would have a degree of $(p-1)/2$, making it computationally infeasible for large $p$. We avoid this by using polynomial modular exponentiation. For polynomials $A(x)$ and $B(x)$, we have that $\gcd(A(x), B(x)) = \gcd(A(x), B(x) \bmod A(x))$. This means we only need to compute $(x + v)^{(p-1)/2} \mod f(x)$ which we do by repeated squaring. In each step, we mutliply the polynomials and immediately finds the remainder when divided by $f(x)$. This ensures that the degree of any intermediate polynomial never exceeds $2 \deg(f(x)) - 2$ which is $2$ in our case.

Let the two square roots of $a$ be $\pm\alpha$. The algorithm attempts to separate them by finding a property that only one root has, which is the value of the Legendre symbol of $(x + v)$. The algorithm succeeds if $(\alpha + v)/p = 1$ and $(-\alpha + v)/p = -1$, or vice-versa, and fails if they cannot be distinguished so $(α + v)/p = (-α + v)/p$.

For a random $v$, the values $\alpha + v$ and $-\alpha + v$ are two distinct non-zero values. The probability that any such value is a quadratic residue is approximately $1/2$. Assuming the choices are roughly independent for large $p$, the probability of failure is
\begin{equation}
    \Pr(\text{Failure}) = \Pr(\text{both are residues}) + \Pr(\text{both are non-residues}) = \frac{1}{2}.
\end{equation}
Hence, the probability of success is also $1/2$. This is a geometric distribution where the probability of success in any single trial is $1/2. The expected number values of $v$ till success is $2$.

*   Tonelli-Shanks:
    *   Deterministic
    *   Arithmetic, it works by finding an element of order $2^\alpha$ and uses it to successively correct a guess for the root.
    *   The complexity depends on the structure of $p-1 = 2^\alpha s$. The runtime is proportional to $\alpha^2$, with Fermat primes the worst case.

*   Cantor-Zassenhaus
    *   Probabilistic
    *   Algebraic, it works by factoring the polynomial $x^2 - a$ into its linear factors $(x-\alpha)(x+\alpha)$.
    *   The complexity is consistently $O(\log^3 p)$, dominated by the polynomial modular exponentiation and does not depend on the structure of $p-1$.

We can also adapt the Cantor-Zassenhaus algorithm to find all roots of a given polynomial. First, we reduce the input polynomial $f(x)$ to a new polynomial $h(x)$ that has the same roots as $f(x)$ but only contains distinct linear factors by computing $h(x) = \gcd(f(x), x^p - x)$. Second, we apply the randomized splitting technique recursively. We split $h(x)$ into two smaller factors and then call the same procedure on those factors until only linear factors remain, which directly give us the roots.

In [88]:
def find_roots_recursive(poly_to_factor, p):
    '''
    Recursively finds roots of a polynomial composed of distinct linear factors.
    '''
    # Constant polynomial (no roots)
    if len(poly_to_factor) <= 1:
        return []

    # Linear polynomial c1*x + c0
    if len(poly_to_factor) == 2:
        c1 = poly_to_factor[1]
        c0 = poly_to_factor[0]
        root = (-c0 * mod_inverse(c1, p) + p) % p
        return [root]

    # Recursive step, split the polynomial
    v = 1
    while True:
        base = [v, 1]  # x + v
        exp = (p - 1) // 2
        pow_poly = poly_pow_mod(base, exp, poly_to_factor, p)
        g_x = list(pow_poly)
        g_x[0] = (g_x[0] - 1 + p) % p
        d_x = poly_gcd(poly_to_factor, g_x, p)

        if 1 < len(d_x) < len(poly_to_factor):
            print(f"  Split {poly_to_string(poly_to_factor)} with v={v} into:")
            other_factor, _ = poly_divmod(poly_to_factor, d_x, p)
            print(f"    Factor 1: {poly_to_string(d_x)}")
            print(f"    Factor 2: {poly_to_string(other_factor)}")
            roots1 = find_roots_recursive(d_x, p)
            roots2 = find_roots_recursive(other_factor, p)
            return sorted(roots1 + roots2)

        v += 1
        if v > 2 * p:
            print(f"Could not split {poly_to_string(poly_to_factor)}.")
            return []

def find_roots(f_x, p):
    '''
    Main function to find the roots of f(x) mod p.
    '''
    print(f"\nFactoring f(x) = {poly_to_string(f_x)}")

    # Reduce f(x) to a product of distinct linear factors via gcd(f(x), x^p - x)
    x_poly = [0, 1]
    xp_mod_f = poly_pow_mod(x_poly, p, f_x, p)
    xp_minus_x = list(xp_mod_f)
    if len(xp_minus_x) < 2: xp_minus_x.extend([0] * (2 - len(xp_minus_x)))
    xp_minus_x[1] = (xp_minus_x[1] - 1 + p) % p
    h_x = poly_gcd(f_x, xp_minus_x, p)

    print(f"  Polynomial with distinct linear factors is h(x) = {poly_to_string(h_x)}")
    # Recursively factor the result
    roots = find_roots_recursive(h_x, p)
    return roots


p = 10708729

# f1(x) = x^4 + 9x^3 + 13x^2 + 2x - 9
f1 = [(-9 + p) % p, 2, 13, 9, 1]

# f2(x) = x^4 + x^3 + x^2 + x + 25
f2 = [25, 1, 1, 1, 1]

# f3(x) = x^4 + x^3 - 10x^2 - 749379x - 120288
f3 = [(-120288 + p) % p, (-749379 + p) % p, (-10 + p) % p, 1, 1]

roots1 = find_roots(f1, p)
print(f"Roots: {roots1}")
for root in roots1:
    result = evaluate_poly(f1, root, p)
    print(f"  f({root}) mod {p} = {result}")

roots2 = find_roots(f2, p)
print(f"Roots: {roots2}")
for root in roots2:
    result = evaluate_poly(f2, root, p)
    print(f"  f({root}) mod {p} = {result}")

roots3 = find_roots(f3, p)
print(f"Roots: {roots3}")
for root in roots3:
    result = evaluate_poly(f3, root, p)
    print(f"  f({root}) mod {p} = {result}")


Factoring f(x) = x^4 + 9x^3 + 13x^2 + 2x + 10708720
  Polynomial with distinct linear factors is h(x) = x^4 + 9x^3 + 13x^2 + 2x + 10708720
  Split x^4 + 9x^3 + 13x^2 + 2x + 10708720 with v=2 into:
    Factor 1: x^2 + 10127754x + 6550660
    Factor 2: x^2 + 580984x + 2199402
  Split x^2 + 10127754x + 6550660 with v=3 into:
    Factor 1: x + 1289936
    Factor 2: x + 8837818
  Split x^2 + 580984x + 2199402 with v=3 into:
    Factor 1: x + 886772
    Factor 2: x + 10402941
Roots: [305788, 1870911, 9418793, 9821957]
  f(305788) mod 10708729 = 0
  f(1870911) mod 10708729 = 0
  f(9418793) mod 10708729 = 0
  f(9821957) mod 10708729 = 0

Factoring f(x) = x^4 + x^3 + x^2 + x + 25
  Polynomial with distinct linear factors is h(x) = x^2 + 8889230x + 6965275
  Split x^2 + 8889230x + 6965275 with v=2 into:
    Factor 1: x + 7049876
    Factor 2: x + 1839354
Roots: [3658853, 8869375]
  f(3658853) mod 10708729 = 0
  f(8869375) mod 10708729 = 0

Factoring f(x) = x^4 + x^3 + 10708719x^2 + 9959350x + 1